In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [6]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [7]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '../nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Users/charlesdecian/Documents/RAG/RAG/rag_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.04747237637639046, 0.021675799041986465, -0.00901806354522705, 0.005356728099286556, 0.02555772289633751, -0.010230298154056072, -0.008413928560912609, 0.03930393233895302, 0.021570468321442604, -0.02409539930522442]


In [13]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [14]:
ids = vector_store.add_documents(documents=all_splits)

In [26]:
ids

['0d05d64c-7475-49de-9a6a-cf0b3e522160',
 '7066e051-2f8a-442c-b492-b886396af00c',
 '197353e1-735a-4327-bd98-3747b135104a',
 'a93e2d0b-f017-4a83-a6ae-ce24902ffe8b',
 'f47bda95-8952-4559-9cdb-cba332812f1c',
 '79f64cf2-f8da-472e-a613-b7f08e919193',
 'a33e2581-61b4-48bd-a57b-dffbb338aa4f',
 '12884b79-9817-45e1-b2b4-f5fc13b66030',
 '82328c4d-0d46-4cea-b674-39ce8b6bb7de',
 'c219ddb7-ad3d-48a3-8c0d-4d669ce30710',
 '2f4df7cb-2b28-428a-819c-e73526277b65',
 '013b7852-2224-4c06-9bcc-06178dbf2152',
 'd46d085d-d8a0-435d-aa24-9cd87cbf74b2',
 'c67b3744-4245-43c8-bd68-edf50af84ebd',
 '9ac00ccd-7dbe-46b6-b98e-05268d85f01c',
 'f8ef2ab4-78e5-4cf4-9567-01bcff629ac3',
 '775708fa-3226-4c3d-a5ba-8b9d79173520',
 '01a04c71-ee33-4a4e-9341-0286e7ef1514',
 'bc88d851-e867-4e27-990c-285f165283e0',
 'e6209e3c-eaeb-4201-8994-f4950fd84d00',
 '3ca2a282-1a9e-4198-868a-c79e67292f5d',
 'b2336d04-0ef6-459b-bd3b-8d66e3e46b4a',
 'd03075bc-494f-40e4-8fd1-102741af4085',
 '86269cca-3c03-4248-bdba-5861b466ef7b',
 'ae6963a2-0b41-

In [17]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[1])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'creationdate': '2023-07-20T16:22:00-04:00', 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'page': 4, 'page_label': '5', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': '../nke-10k-2023.pdf', 'start_index': 3125, 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'title': '0000320187-23-000039', 'total_pages': 107}


In [21]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'creationdate': '2023-07-

In [23]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[1]
print(f"Score: {score}\n")
print(doc)

Score: 0.39114248752593994

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' meta

In [25]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=2)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='35b2394a-94fb-4acf-a2a9-99a51138c871', metadata={'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'creationdate': '2023-07-20T16:22:00-04:00', 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'page': 26, 'page_label': '27', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': '../nke-10k-2023.pdf', 'start_index': 804, 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'title': '0000320187-23-000039', 'total_pages': 107}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of wh

# Mieux 

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

VectorStoreRetriever supports search types of "similarity" (default), "mmr" (maximum marginal relevance, described above), and "similarity_score_threshold".